In [92]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import display

# %matplotlib inline

from pathlib import Path


In [77]:
def collect_one(run, csv_folder):
    with open(run / 'config.json', 'r') as f:
        cfg = json.load(f)
        
    run_dir = cfg["run_dir"]
    parameters = {p.split('=')[0]: p.split('=')[1] for p in run_dir.split('/') if "=" in p}

    data = {}
    for yamlf in (run / 'inference' / csv_folder).rglob('*.csv'):
        data[yamlf.parent.stem] = pd.read_csv(yamlf)
    
    data = pd.concat(data, axis=1)
    data.columns.names = ['type', 'metric']
    data.sort_values(by=['type', 'metric'], axis=1, inplace=True)
    data.drop(columns=['epoch', 'step'], level='metric', inplace=True)
    
    if data.empty:
        print(f'Pred folder is empty: {csv_folder}')
    
    for k, v in parameters.items():
        data[k] = v
    
    return data

def collect_all(root, csv_folder):
    root = Path(root)
    metrics = [collect_one(csvf.parents[1], csvf.name) for csvf in list(root.rglob(csv_folder))]
    metrics = pd.concat(metrics, ignore_index=True)
    return metrics

default_fields_dict = {
    'r1': lambda x: u"{:.1f}".format(x),
    'r5': lambda x: u"{:.1f}".format(x),
    'r10': lambda x: u"{:.1f}".format(x),
    'meanr': lambda x: u"{:.1f}".format(x),
    'medr': lambda x: int(x),
    'spice': lambda x: u"{:.3f}".format(x),
    'spacy': lambda x: u"{:.3f}".format(x),
}
def render_to_latex(metrics, rename_func=default_fields_dict, **latex_kwargs):
    m = metrics.copy()
    # renaming
    for col, lambda_fn in rename_func.items():
        m[col] = m[col].apply(lambda_fn)
    # m = m.applymap(lambda x: u"{:.2f}".format(x))
    ltex = m.style.to_latex(
        **latex_kwargs
    )
    return ltex

In [123]:
# Compute metrics for each detected run

def summarize_metrics(
        metrics, 
        dataset=None, 
        model=None,
        translator=None,
        tok_position=None, 
        training_setup=None, 
        loss=None, 
        learning_rate=None, 
        finetuning=None, 
        drop_i2t=True):
    
    if dataset is not None:
        metrics = metrics[(metrics['data'] == dataset)]
        metrics.drop(columns="data", inplace=True)

    # TODO: as of now, there is only one split seed.
    # In the future, we would have to average among different splits
    # metrics.drop(columns="split_seed", inplace=True)

    id_vars = ['data', 'model', 'translator', 'tok_position', 'training-setup', 'loss', 'lr', 'finetuning']

    if translator is not None:
        metrics = metrics[metrics['translator'].isin(learning_rate)]
        if len(translator) == 1:
            metrics.drop(columns="translator", inplace=True)
            id_vars.remove('translator')
    if model is not None:
        metrics = metrics[metrics['model'].isin(learning_rate)]
        if len(model) == 1:
            metrics.drop(columns="model", inplace=True)
            id_vars.remove('model')
    if learning_rate is not None:
        metrics = metrics[metrics['lr'].isin(learning_rate)]
        if len(learning_rate) == 1:
            metrics.drop(columns="lr", inplace=True)
            id_vars.remove('lr')
    if finetuning is not None:
        metrics = metrics[metrics['finetuning'].isin(finetuning)]
        if len(finetuning) == 1:
            metrics.drop(columns="finetuning", inplace=True)
            id_vars.remove('finetuning')
    if tok_position is not None:
        metrics = metrics[metrics['tok_position'].isin(tok_position)]
        if len(tok_position) == 1:
            metrics.drop(columns="tok_position", inplace=True)
            id_vars.remove('tok_position')
    if training_setup is not None:
        metrics = metrics[metrics['training-setup'].isin(training_setup)]
        if len(training_setup) == 1:
            metrics.drop(columns="training-setup", inplace=True)
            id_vars.remove('training-setup')
    if loss is not None:
        metrics = metrics[metrics['loss'].isin(loss)]
        if len(loss) == 1:
            metrics.drop(columns="loss", inplace=True)
            id_vars.remove('loss')

    if drop_i2t:
        # remove columns containing i2t in the name of the second level of the multiindex
        metrics = metrics.loc[:, ~metrics.columns.get_level_values(1).str.contains('i2t')]

    metrics.set_index(id_vars, inplace=True)
    # split into different dataframes, one for each column (first level of the multiindex)
    column_types = list(metrics.columns.get_level_values(0).unique())
    metrics = {c: metrics.loc[:, c].copy() for c in column_types}
    {k: v.columns.set_names(k, inplace=True) for k, v in metrics.items()}
    return metrics

In [4]:
# rename content of the table
def rename_fn(v):
    mapping = {'ContrastiveFixed': 'Triplet',
               'InfoNCELoss': 'InfoNCE'}
    if v in mapping:
        return mapping[v]
    return v

# Results (best contrastive sum checkpoint)

In [124]:
# collect all data on kit
ROOT = "runs"

metrics = collect_all(ROOT, 'best-contrastive-sum')
metrics = summarize_metrics(
    metrics,
    # tok_position=["tok_beginning"],
    melt=False
    )

for k in metrics:
    display(metrics[k])

entities_retrieval                                                                                                      entities_sum  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                    
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled           1.188381   
                                                                                                         shallow-vpt-5      1.354643   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled           1.016644   
                                                                                                         shallow-vpt-5      1.209226   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled           1.303040   
                                                                                                         shallow-vpt-5      1.258577   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled           1.156941   
                                                                                                         shallow-vpt-5      1.171733   

entities_retrieval                                                                                                      entity-kmin-r@1  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                       
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled              0.148045   
                                                                                                         shallow-vpt-5         0.184358   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled              0.108939   
                                                                                                         shallow-vpt-5         0.173184   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled              0.173184   
                                                                                                         shallow-vpt-5         0.175978   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled              0.148045   
                                                                                                         shallow-vpt-5         0.162011   

entities_retrieval                                                                                                      entity-kmin-r@10  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                        
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled               0.126609   
                                                                                                         shallow-vpt-5          0.138970   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled               0.108978   
                                                                                                         shallow-vpt-5          0.125708   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled               0.134637   
                                                                                                         shallow-vpt-5          0.124807   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled               

general_retrieval_tok_beginning                                                                                         contrastive_sum  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                       
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled              5.730795   
                                                                                                         shallow-vpt-5         5.614545   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled              5.710568   
                                                                                                         shallow-vpt-5         5.749318   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled              5.615455   
                                                                                                         shallow-vpt-5         5.549318   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled              5.688636   
                                                                                                         shallow-vpt-5         5.655341   

general_retrieval_tok_beginning                                                                                          t2i-r@1  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled       0.397614   
                                                                                                         shallow-vpt-5  0.417727   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.405227   
                                                                                                         shallow-vpt-5  0.432500   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled       0.386250   
                                                                                                         shallow-vpt-5  0.410000   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.402386   
                                                                                                         shallow-vpt-5  0.421136   

general_retrieval_tok_beginning                                                                                         t2i-r@10  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled       0.819773   
                                                                                                         shallow-vpt-5  0.799886   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.815455   
                                                                                                         shallow-vpt-5  0.816023   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled       0.785795   
                                                                                                         shallow-vpt-5  0.788409   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.805227   
                                                                                                    

general_retrieval_tok_in_place                                                                                          contrastive_sum  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                       
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled              5.703864   
                                                                                                         shallow-vpt-5         5.603295   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled              5.673750   
                                                                                                         shallow-vpt-5         5.727273   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled              5.476136   
                                                                                                         shallow-vpt-5         5.555000   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled              5.647159   
                                                                                                         shallow-vpt-5         5.653068   

general_retrieval_tok_in_place                                                                                           t2i-r@1  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled       0.395341   
                                                                                                         shallow-vpt-5  0.418295   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.399545   
                                                                                                         shallow-vpt-5  0.428864   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled       0.307841   
                                                                                                         shallow-vpt-5  0.360114   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.357841   
                                                                                                         shallow-vpt-5  0.397614   

general_retrieval_tok_in_place                                                                                          t2i-r@10  \
data                     model  translator  tok_position  training-setup                  loss     lr    finetuning                
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place  with_entities                   info-nce 5e-05 disabled       0.814886   
                                                                                                         shallow-vpt-5  0.802500   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.811932   
                                                                                                         shallow-vpt-5  0.813977   
                                            tok_beginning with_entities                   info-nce 5e-05 disabled       0.760000   
                                                                                                         shallow-vpt-5  0.777841   
                                                          with_entities_and_only_entities info-nce 5e-05 disabled       0.788295   
                                                                                                    